<a href="https://colab.research.google.com/github/armarmarm927/dl_lecture_competition_pub/blob/main/sparkproto22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomAffine([-10,10],scale=(1,1.25)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))
])

In [5]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

100%|██████████| 170M/170M [00:38<00:00, 4.38MB/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
model_ft = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model_ft.fc = nn.Linear(model_ft.fc.in_features, 10)
net = model_ft.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 172MB/s]


In [7]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.SGD(net.parameters(),lr=0.1,momentum=0.9,weight_decay=0.00005)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60,120,160], gamma=0.2)

In [8]:
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
num_epochs = 200
loss_list = []
acc_list = []
train_acc_list = []

for epoch in range(num_epochs):
    net.train()
    correct_train = 0
    total_train = 0
    epoch_loss = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        inputs, targets_a, targets_b, lam = mixup_data(inputs, labels, alpha=1.0)

        optimizer.zero_grad()
        outputs = net(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    train_accuracy = 100. * correct_train / total_train
    train_acc_list.append(train_accuracy)
    print(f'Epoch {epoch+1}: Loss: {epoch_loss/len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')
    loss_list.append(epoch_loss/len(train_loader))

    net.eval()
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    test_accuracy = 100. * correct / total
    acc_list.append(test_accuracy)
    print(f'Test Accuracy: {test_accuracy:.2f}%')

    # 混同行列の出力
    cm = confusion_matrix(y_true, y_pred)
    print(f'Confusion Matrix (Epoch {epoch+1}):')
    print(cm)

    scheduler.step()
    torch.save(net.state_dict(), '/content/drive/MyDrive/Weight_Epoch50.pth')

Epoch 1: Loss: 2.1109, Train Accuracy: 22.03%
Test Accuracy: 49.00%
Confusion Matrix (Epoch 1):
[[771  37 109   3   1  13   8   8  30  20]
 [129 756  10   6   1   7  10  14  19  48]
 [134  24 358  24  61 183 175  22  10   9]
 [ 39  16 159  79  27 560  51  22  28  19]
 [ 78  13 176   7 230 186 154 141   8   7]
 [ 21  10 129  23  22 726  29  24  12   4]
 [ 16  10 127  88  69 122 537  13  10   8]
 [ 36  11  99   5  31 239   6 546   3  24]
 [437  65  32   4   0  14   3   0 429  16]
 [143 293  18   6   1  12   8  28  23 468]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 2: Loss: 1.8813, Train Accuracy: 33.37%
Test Accuracy: 60.65%
Confusion Matrix (Epoch 2):
[[801  11  19   2   1   2   3   5 106  50]
 [ 28 619   2   0   0   1   0   0  94 256]
 [229  24 410  17  76  51 110  31  16  36]
 [ 90  29  86 248  27 229 106  39  50  96]
 [138  16 128  31 351  18 131  98  33  56]
 [ 44  15  70  71  23 594  22  82  16  63]
 [ 36  27  69  27  20  27 714   5  22  53]
 [119  20  51   7  50  38   9 593   5 108]
 [ 80  10   4   2   1   3   1   0 866  33]
 [ 35  37   3   2   0   0   1   1  52 869]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 3: Loss: 1.7749, Train Accuracy: 35.82%
Test Accuracy: 65.94%
Confusion Matrix (Epoch 3):
[[613  60 111   1   0   9   3  38  99  66]
 [  2 885   4   0   0   2   1   4  23  79]
 [ 48  19 620  17  42  95  40  90  12  17]
 [ 15  27  86 209  18 396  21 151  41  36]
 [ 46  18  98  39 357  46  37 333  14  12]
 [  5  13  57  31  17 691   5 148  13  20]
 [  4  44  91  60  37  74 614  27  23  26]
 [  4  10  18  11   0  48   5 890   2  12]
 [ 43  42   3   1   1   5   1   6 865  33]
 [ 14  86   5   2   0   2   0  22  19 850]]


...

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 116: Loss: 1.3517, Train Accuracy: 46.48%
Test Accuracy: 89.33%
Confusion Matrix (Epoch 116):
[[951   4   8   4   2   2   0   4  16   9]
 [  5 971   2   4   0   0   1   0   6  11]
 [ 31   2 892  24  15  12  10  11   3   0]
 [ 11   4  30 803  23  94  21   9   4   1]
 [ 10   1  27  23 870  21  12  34   1   1]
 [ 10   0  19 110  19 816   7  15   2   2]
 [ 12   3  35  27   6  10 896   6   3   2]
 [  8   1   8  21  12  27   1 919   1   2]
 [ 35  17   4   4   1   0   0   2 928   9]
 [ 25  59   4   8   0   3   1   3  10 887]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 117: Loss: 1.3592, Train Accuracy: 46.92%
Test Accuracy: 89.91%
Confusion Matrix (Epoch 117):
[[919   8  14   3   2   2   1   4  37  10]
 [  1 962   4   4   0   1   0   0  11  17]
 [ 22   1 912  18  15  16   8   6   2   0]
 [ 10   3  27 775  17 130  22   7   6   3]
 [  4   1  34  26 881  24  16  13   1   0]
 [  8   1  20  77  20 855   7  10   1   1]
 [  5   1  32  29   6   7 919   0   1   0]
 [  6   0  10  14  22  27   2 914   2   3]
 [ 15  10   6   5   1   1   1   1 958   2]
 [ 14  52   4   7   0   2   1   1  23 896]]


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
plt.plot(acc_list)
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()

In [1]:
plt.plot(train_acc_list)
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.plot(loss_list)
plt.title('Cross Entropy Loss')
plt.xlabel('Epoch')
plt.xticks(np.arange(0,200,20))
plt.ylabel('Accuracy')
plt.show()